In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import numpy as np
import sklearn

from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
from sklearn.decomposition import SparseCoder, sparse_encode
from sklearn.linear_model import orthogonal_mp_gram

import scipy.linalg as LA
from scipy.sparse.linalg import lsqr, lsmr

In [2]:
eye = np.eye(3)

In [3]:
eye

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [4]:
eye[0:1]

array([[1., 0., 0.]])

In [5]:
eye[0:1].shape

(1, 3)

In [6]:
eye[0:1].T.shape

(3, 1)

In [7]:
eye[[0, 1]]

array([[1., 0., 0.],
       [0., 1., 0.]])

# Solve

In [8]:
def OMP(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    # gram = D @ D.T
    # cov = D @ Y.T
    # X = sparse_encode(Y, 
    #                   D, 
    #                   algorithm='omp', 
    #                   n_nonzero_coefs = T_0,
    #                   gram = gram,
    #                   cov = cov,
    #                   copy_cov=False)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            # gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsd')
            # gamma, istop, itn, res = lsmr(D_I[:j+1].T, y)[:4]
            gamma = np.linalg.solve(D_I[:j+1] @ D_I[:j+1].T, D_I[:j+1] @ y)
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - D_I[:(j+1)].T @ gamma 
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

# Handwritten QR (Gram-Schmidt)

In [9]:
def OMP_2(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0
        Q = np.empty((D.shape[1], T_0))
        R = np.empty((T_0, T_0))
        
        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]

            if j == 0:
                # Q, R = LA.qr(D[k][:, np.newaxis],
                #             overwrite_a = False,
                #             mode = 'economic')
                D_k_norm = LA.norm(D[k])
                R[0, 0] = D_k_norm
                Q[:, 0] = D[k] / D_k_norm
                
                if debug:
                    print(LA.norm(D[k]))
                    print(Q[:, 0].T @ Q[:, 0])
                    print(R)

                gamma = (Q[:, :j+1].T @ y) / D_k_norm
                
            else:
                # Q, R = LA.qr_insert(Q, R, D[k][:, np.newaxis], j, which='col') 
                dot = Q[:, :j].T @ D[k]
                R[0:j, j] = dot
                q_j = D[k] - Q[:, :j] @ dot

                q_j_norm = LA.norm(q_j)
                R[j, j] = q_j_norm
                Q[:, j] = q_j / q_j_norm

                if debug:            
                    print(Q[:, :j+1].T.shape)
                    print(y.shape)
                gamma = LA.solve_triangular(R[:j+1, :j+1], 
                                        Q[:, :j+1].T @ y, 
                                        overwrite_b = True,
                                        check_finite = False)
                

            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

## Using numpy's norm instead of scipy's

In [10]:
def OMP_3(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0
        Q = np.empty((D.shape[1], T_0))
        R = np.empty((T_0, T_0))
        
        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]

            if j == 0:
                # Q, R = LA.qr(D[k][:, np.newaxis],
                #             overwrite_a = False,
                #             mode = 'economic')
                D_k_norm = np.linalg.norm(D[k])
                R[0, 0] = D_k_norm
                Q[:, 0] = D[k] / D_k_norm
                
                if debug:
                    print(np.linalg.norm(D[k]))
                    print(Q[:, 0].T @ Q[:, 0])
                    print(R)

                gamma = (Q[:, :j+1].T @ y) / D_k_norm
                
            else:
                # Q, R = LA.qr_insert(Q, R, D[k][:, np.newaxis], j, which='col') 
                dot = Q[:, :j].T @ D[k]
                R[0:j, j] = dot
                q_j = D[k] - Q[:, :j] @ dot

                q_j_norm = np.linalg.norm(q_j)
                R[j, j] = q_j_norm
                Q[:, j] = q_j / q_j_norm

                if debug:            
                    print(Q[:, :j+1].T.shape)
                    print(y.shape)
                gamma = LA.solve_triangular(R[:j+1, :j+1], 
                                        Q[:, :j+1].T @ y, 
                                        overwrite_b = True,
                                        check_finite = False)
                

            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

## precomputing the dictionary norms

In [11]:
def OMP_4(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))
    D_norm = np.linalg.norm(D, axis=1)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0
        Q = np.empty((D.shape[1], T_0))
        R = np.empty((T_0, T_0))
        
        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]

            if j == 0:
                # Q, R = LA.qr(D[k][:, np.newaxis],
                #             overwrite_a = False,
                #             mode = 'economic')
                # D_k_norm = np.linalg.norm(D[k])
                D_k_norm = D_norm[k]
                R[0, 0] = D_k_norm
                Q[:, 0] = D[k] / D_k_norm
                
                if debug:
                    print(np.linalg.norm(D_k_norm))
                    print(Q[:, 0].T @ Q[:, 0])
                    print(R)

                gamma = (y @ Q[:, :j+1]) / D_k_norm
                
            else:
                # Q, R = LA.qr_insert(Q, R, D[k][:, np.newaxis], j, which='col') 
                dot = Q[:, :j].T @ D[k]
                R[0:j, j] = dot
                q_j = D[k] - Q[:, :j] @ dot

                q_j_norm = np.linalg.norm(q_j)
                R[j, j] = q_j_norm
                Q[:, j] = q_j / q_j_norm

                if debug:            
                    print(Q[:, :j+1].T.shape)
                    print(y.shape)
                gamma = LA.solve_triangular(R[:j+1, :j+1], 
                                        Q[:, :j+1].T @ y, 
                                        overwrite_b = True,
                                        check_finite = False)
                

            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

In [12]:
def OMP_verif(code):
    print(code)
    I = np.nonzero(code)
    print(I)
    print(code[I])

In [13]:
X, y = make_regression(n_samples = 50, n_features = 20, n_targets = 2, noise=4, random_state=0)

In [14]:
X.shape

(50, 20)

In [15]:
y.shape

(50, 2)

In [16]:
b_numpy = OMP(y.T, 3, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.071243986
(2,)
(50, 2)
(50,)
1675390.9154018096
(3,)
(50, 3)
(50,)
1347560.0488303776
(1,)
(50, 1)
(50,)
1817892.1699581572
(2,)
(50, 2)
(50,)
1455231.954008666
(3,)
(50, 3)
(50,)
1157900.6749918351


In [17]:
OMP_verif(b_numpy)

[[  0.         132.14938371   0.           0.           0.
    0.           0.           0.          70.15323381   0.
    0.          98.72352887   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         136.86874232   0.           0.           0.
    0.           0.           0.          75.95811465   0.
    0.           0.           0.           0.           0.
    0.          67.35538119   0.           0.           0.        ]]
(array([0, 0, 0, 1, 1, 1]), array([ 1,  8, 11,  1,  8, 16]))
[132.14938371  70.15323381  98.72352887 136.86874232  75.95811465
  67.35538119]


In [18]:
b_scipy = OMP_2(y.T, 3, X.T, debug=True)

7.633996503399396
0.9999999999999999
[[ 7.6339965   3.7894417   1.49588522]
 [ 3.7894417  67.05616075 -2.05742687]
 [ 1.49588522 -2.05742687 65.66658662]]
(1,)
(50, 1)
(50,)
2449425.071243986
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1675390.9154018096
(3, 50)
(50,)
(3,)
(50, 3)
(50,)
1347560.0488303776
6.338597587156968
1.0
[[ 6.33859759  3.7894417   1.49588522]
 [ 3.7894417  67.05616075  2.05742687]
 [ 1.49588522  2.05742687 65.66658662]]
(1,)
(50, 1)
(50,)
1817892.1699581572
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1455231.954008666
(3, 50)
(50,)
(3,)
(50, 3)
(50,)
1157900.6749918351


In [19]:
OMP_verif(b_scipy)

[[  0.         132.14938371   0.           0.           0.
    0.           0.           0.          70.15323381   0.
    0.          98.72352887   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         136.86874232   0.           0.           0.
    0.           0.           0.          75.95811465   0.
    0.           0.           0.           0.           0.
    0.          67.35538119   0.           0.           0.        ]]
(array([0, 0, 0, 1, 1, 1]), array([ 1,  8, 11,  1,  8, 16]))
[132.14938371  70.15323381  98.72352887 136.86874232  75.95811465
  67.35538119]


In [20]:
b_scipy2 = OMP_3(y.T, 3, X.T, debug=True)

7.633996503399396
0.9999999999999999
[[ 7.6339965   0.59783598  0.23599624]
 [ 3.7894417   8.16693044 -0.26919709]
 [ 1.49588522  2.05742687  8.0955806 ]]
(1,)
(50, 1)
(50,)
2449425.071243986
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1675390.9154018096
(3, 50)
(50,)
(3,)
(50, 3)
(50,)
1347560.0488303776
6.338597587156968
1.0
[[6.33859759 0.59783598 0.23599624]
 [3.7894417  8.16693044 0.26919709]
 [1.49588522 2.05742687 8.0955806 ]]
(1,)
(50, 1)
(50,)
1817892.1699581572
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1455231.954008666
(3, 50)
(50,)
(3,)
(50, 3)
(50,)
1157900.6749918351


In [21]:
OMP_verif(b_scipy2)

[[  0.         132.14938371   0.           0.           0.
    0.           0.           0.          70.15323381   0.
    0.          98.72352887   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         136.86874232   0.           0.           0.
    0.           0.           0.          75.95811465   0.
    0.           0.           0.           0.           0.
    0.          67.35538119   0.           0.           0.        ]]
(array([0, 0, 0, 1, 1, 1]), array([ 1,  8, 11,  1,  8, 16]))
[132.14938371  70.15323381  98.72352887 136.86874232  75.95811465
  67.35538119]


In [22]:
b_scipy3 = OMP_4(y.T, 3, X.T, debug=True)

7.633996503399395
1.0000000000000002
[[ 7.6339965   0.59783598  0.23599624]
 [ 3.7894417   8.16693044 -0.26919709]
 [ 1.49588522  2.05742687  8.0955806 ]]
(1,)
(50, 1)
(50,)
2449425.0712439856
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1675390.9154018096
(3, 50)
(50,)
(3,)
(50, 3)
(50,)
1347560.0488303776
6.3385975871569675
1.0000000000000004
[[6.33859759 0.59783598 0.23599624]
 [3.7894417  8.16693044 0.26919709]
 [1.49588522 2.05742687 8.0955806 ]]
(1,)
(50, 1)
(50,)
1817892.1699581577
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1455231.9540086659
(3, 50)
(50,)
(3,)
(50, 3)
(50,)
1157900.6749918351


In [23]:
OMP_verif(b_scipy3)

[[  0.         132.14938371   0.           0.           0.
    0.           0.           0.          70.15323381   0.
    0.          98.72352887   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         136.86874232   0.           0.           0.
    0.           0.           0.          75.95811465   0.
    0.           0.           0.           0.           0.
    0.          67.35538119   0.           0.           0.        ]]
(array([0, 0, 0, 1, 1, 1]), array([ 1,  8, 11,  1,  8, 16]))
[132.14938371  70.15323381  98.72352887 136.86874232  75.95811465
  67.35538119]


In [24]:
# b_scipy4 = OMP_5(y.T, 3, X.T, debug=True)

In [25]:
# OMP_verif(b_scipy4)

In [26]:
np.allclose(b_scipy[np.nonzero(b_scipy)], b_numpy[np.nonzero(b_numpy)])

True

In [27]:
np.allclose(b_scipy2[np.nonzero(b_scipy2)], b_numpy[np.nonzero(b_numpy)])

True

In [28]:
np.allclose(b_scipy3[np.nonzero(b_scipy3)], b_numpy[np.nonzero(b_numpy)])

True

In [29]:
# np.allclose(b_scipy4[np.nonzero(b_scipy4)], b_numpy[np.nonzero(b_numpy)])

## IT WORKS !!!

In [30]:
X, y = make_regression(n_samples = 50, n_features = 300, n_targets = 20_000, noise=4, random_state=0)

In [31]:
%timeit OMP(y.T, 1, X.T)

837 ms ± 27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%timeit OMP_2(y.T, 1, X.T)

781 ms ± 61.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit OMP_3(y.T, 1, X.T)

728 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit OMP_4(y.T, 1, X.T)

585 ms ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
# %timeit OMP_5(y.T, 1, X.T)

In [36]:
%load_ext line_profiler
from line_profiler import profile

In [37]:
np_OMP = profile(OMP)
%lprun -f np_OMP np_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.3797 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_8792\3367727764.py
Function: OMP at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        393.0    393.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7                                               # gram = D @ D.T
     8                                               # cov = D @ Y.T
     9                                               # X = sparse_encode(Y, 
    10                                               #                   D, 
    11                                               #   

In [38]:
sci_OMP = profile(OMP_2)
%lprun -f sci_OMP sci_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.18994 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_8792\2337247931.py
Function: OMP_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_2(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        322.0    322.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7     20001     172167.0      8.6      1.4      for i, y in enumerate(Y):
     8     20000      54264.0      2.7      0.5          I = []
     9     20000     350837.0     17.5      2.9          D_I = np.zeros((T_0, D.shape[1]))
    10     20000      42722.0      2.1      0.4          r = y
    11     20000      43406.0      2.2      0.4 

In [39]:
sci_OMP2 = profile(OMP_3)
%lprun -f sci_OMP2 sci_OMP2(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.03486 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_8792\3915918126.py
Function: OMP_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_3(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        444.0    444.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7     20001     166209.0      8.3      1.6      for i, y in enumerate(Y):
     8     20000      62375.0      3.1      0.6          I = []
     9     20000     323009.0     16.2      3.1          D_I = np.zeros((T_0, D.shape[1]))
    10     20000      43380.0      2.2      0.4          r = y
    11     20000      41546.0      2.1      0.4 

In [40]:
sci_OMP3 = profile(OMP_4)
%lprun -f sci_OMP3 sci_OMP3(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 0.80585 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_8792\2243887603.py
Function: OMP_4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_4(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1       1695.0   1695.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6         1       1114.0   1114.0      0.0      D_norm = np.linalg.norm(D, axis=1)
     7                                           
     8     20001     158533.0      7.9      2.0      for i, y in enumerate(Y):
     9     20000      57125.0      2.9      0.7          I = []
    10     20000     291904.0     14.6      3.6          D_I = np.zeros((T_0, D.shape[1]))
    11     20000      4

In [41]:
# sci_OMP4 = profile(OMP_5)
# %lprun -f sci_OMP4 sci_OMP4(y.T, 1, X.T)

In [42]:
X1, y1 = make_regression(n_samples = 50, n_features = 20, n_targets = 100, noise=4, random_state=0)

In [43]:
X2, y2 = make_regression(n_samples = 50, n_features = 20, n_targets = 100, noise=4, random_state=0)

In [44]:
X1.shape

(50, 20)

In [45]:
X2.shape

(50, 20)

In [46]:
X_12 = np.stack([X1, X2], axis=0)

In [47]:
X_12.shape

(2, 50, 20)

In [48]:
y_12 = np.stack([y1, y2], axis=0)

In [49]:
LA.lstsq(X_12, y_12)[0].shape

(2, 20, 100)

In [50]:
np.eye(3)[0][:, np.newaxis].shape

(3, 1)